<a href="https://colab.research.google.com/github/kyuchani/OSSProtopyte.ipynb/blob/main/OSSProtopyte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch
import os

# 1. Hugging Face 토큰 설정 확인 (필요시)
# (Colab Secrets에 HF_TOKEN이 설정되어 있다면 이 부분은 건너뛰어도 됩니다)
# from huggingface_hub import notebook_login
# notebook_login() # 이미 로그인되어 있거나 퍼블릭 모델이면 스킵 가능

# 2. BitsAndBytes 설정 (4bit 양자화)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 # 또는 torch.float16
)

# 3. 모델 및 토크나이저 로딩
model_id = "iRASC/BioLlama-Ko-8B" # 모델 ID를 정확히 확인합니다.
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto" # 여기서 이미 디바이스가 결정됨
)
print("모델 로드 완료!")

# 4. 텍스트 생성 파이프라인 준비
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16, # 모델 로딩 시 사용한 compute_dtype과 일치
    # device=0 # 이 라인을 삭제합니다!
)
print("Generator 파이프라인 준비 완료!")

# 이제 RAG 추론 코드를 이어서 실행하면 됩니다.

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Device set to use cuda:0


모델 로드 완료!
Generator 파이프라인 준비 완료!


In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer
import os # 파일 시스템 경로 관리를 위해 os 모듈 추가

# 1. 예시 문서 준비 (의료 관련 내용으로 변경)
documents = [
    "독감은 인플루엔자 바이러스에 의해 발생하는 급성 호흡기 질환입니다. 주요 증상으로는 고열, 기침, 인후통, 근육통 등이 있습니다.",
    "고혈압은 혈압이 비정상적으로 높은 상태를 의미하며, 심장병, 뇌졸중, 신장 질환의 주요 위험 인자입니다. 규칙적인 운동과 저염식이가 중요합니다.",
    "당뇨병은 혈당 수치가 높아지는 만성 질환으로, 인슐린 부족 또는 인슐린 저항성으로 인해 발생합니다. 합병증 예방을 위해 혈당 관리가 필수적입니다.",
    "아스피린은 해열, 진통, 소염 작용을 하는 비스테로이드성 소염진통제(NSAID)입니다. 혈전 생성을 억제하는 효과도 있어 심혈관 질환 예방에도 사용됩니다.",
    "MRI는 자기장을 이용하여 인체 내부를 촬영하는 의료 영상 기술입니다. X선 노출이 없어 방사선 위험이 없습니다.",
    "항생제는 세균 감염을 치료하는 데 사용되는 약물입니다. 바이러스 감염에는 효과가 없으며, 오남용 시 내성을 유발할 수 있습니다.",
    "암은 비정상적인 세포 증식으로 인해 발생하는 질병입니다. 조기 진단과 치료가 중요하며, 수술, 방사선 치료, 항암 화학요법 등이 있습니다.",
    "백신은 특정 질병에 대한 면역력을 미리 형성하여 감염을 예방하는 의약품입니다. 예방 접종을 통해 집단 면역을 형성하는 데 기여합니다.",

    # 붉은사슴 뿔버섯 및 독성 관련 정보: **인과 관계를 직접 명시**
    "붉은사슴 뿔버섯은 치명적인 맹독성 버섯입니다. **붉은사슴 뿔버섯을 섭취하면 장염이 발생할 수 있습니다.**", # 가장 직접적인 인과 관계 명시
    "붉은사슴 뿔버섯에는 인체에 매우 치명적인 독소(마이코톡신)가 포함되어 있습니다.",
    "**붉은사슴 뿔버섯 섭취 시, 독소로 인해 급성 장염 증상(복통, 설사, 구토, 발열 등)이 나타납니다.**", # 증상과 원인 명확히 연결
    "**급성 장염 증상은 장염의 한 종류이며, 붉은사슴 뿔버섯과 같은 독성 물질 섭취로 인해 유발됩니다.**", # '급성 장염 증상'과 '장염'의 연결 강조
    "붉은사슴 뿔버섯의 독소는 소화기 계통에 심각한 손상을 일으켜 **직접적으로 장염을 유발**합니다.", # '직접적으로 유발' 강조
    "이 버섯 중독은 단순 장염 증상뿐만 아니라, 백혈구 감소증, 신부전 등 심각한 전신 증상을 초래하며 사망에 이를 수도 있습니다.",
    "붉은사슴 뿔버섯 중독에는 특정 해독제가 없으며, 증상에 대한 대증 요법과 생명 유지가 유일한 치료법입니다.",
    "붉은사슴 뿔버섯은 붉은색 또는 주황색의 뿔 모양을 하고 있으며, 식용 버섯과 생김새가 달라 구별이 가능합니다.",

    # 장염 관련 정보: 독소와의 연결 강조
    "장염은 위와 소장의 염증을 의미합니다. **장염은 세균, 바이러스, 독소, 기생충 또는 특정 식품 섭취 등으로 발생할 수 있습니다.**",
    "**독소 섭취는 장염 발생의 명확한 원인 중 하나입니다.** 독소로 인한 장염은 증상이 빠르게 나타나는 특징이 있습니다."
]
chunked_documents = documents

# 2. 임베딩 모델 로드 (이 부분은 변경 없음)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("임베딩 모델 로드 완료!")

# 3. ChromaDB 클라이언트 초기화 및 컬렉션 생성
# --- 수정된 부분: ChromaDB를 파일 시스템에 저장하도록 path 지정 ---
chroma_db_path = "./my_chroma_db"
# 디렉토리가 없으면 생성
if os.makedirs(chroma_db_path, exist_ok=True): # exist_ok=True로 변경하여 이미 있어도 오류 발생 안함
    print(f"ChromaDB 저장 경로 '{chroma_db_path}' 생성 또는 이미 존재함.")
client = chromadb.PersistentClient(path=chroma_db_path) # PersistentClient 사용
# -------------------------------------------------------------------
collection_name = "my_rag_documents"

# 기존 컬렉션 삭제 (재색인할 때만 필요, 데이터 변경 시마다 실행)
try:
    client.delete_collection(name=collection_name)
    print(f"기존 컬렉션 '{collection_name}' 삭제 완료.")
except Exception as e:
    print(f"컬렉션 삭제 중 오류 발생 또는 컬렉션 없음: {e}")

collection = client.create_collection(name=collection_name)
print(f"ChromaDB 컬렉션 '{collection_name}' 생성 완료.")

# 4. 문서 임베딩 및 벡터 데이터베이스에 추가
ids = [f"doc_{i}" for i in range(len(chunked_documents))]
embeddings = embedding_model.encode(chunked_documents).tolist()

collection.add(
    documents=chunked_documents,
    embeddings=embeddings,
    ids=ids
)
print(f"총 {len(chunked_documents)}개의 의료 문서가 ChromaDB에 추가되었습니다.")
print("새로운 의료 데이터로 색인 단계 완료!")

# --- 디버깅 단계 추가: ChromaDB 내용 직접 확인 ---
print("\n--- ChromaDB 내용 직접 확인 (디버깅) ---")
test_query = "붉은사슴 뿔버섯은 장염을 일으키는가?" # 직접적인 질문으로 테스트
test_query_embedding = embedding_model.encode([test_query]).tolist()
test_results = collection.query(
    query_embeddings=test_query_embedding,
    n_results=1,
    include=['documents', 'distances']
)

if test_results and test_results['documents'] and test_results['documents'][0]:
    print(f"'{test_query}' 검색 결과 (DB 내부 직접 쿼리):")
    print(f"문서: {test_results['documents'][0][0]}")
    print(f"유사도 거리: {test_results['distances'][0][0]:.4f}")
else:
    print(f"'{test_query}'에 대한 문서를 ChromaDB에서 찾을 수 없습니다. (디버깅 실패)")
print("---------------------------------------")

# 이 아래에 LLM 로드 코드가 위치해야 합니다. (이전 답변에서 제공됨)
# 그 다음에 RAG 대화 루프 코드가 위치합니다. (이전 답변에서 제공됨)

임베딩 모델 로드 완료!
기존 컬렉션 'my_rag_documents' 삭제 완료.
ChromaDB 컬렉션 'my_rag_documents' 생성 완료.
총 18개의 의료 문서가 ChromaDB에 추가되었습니다.
새로운 의료 데이터로 색인 단계 완료!

--- ChromaDB 내용 직접 확인 (디버깅) ---
'붉은사슴 뿔버섯은 장염을 일으키는가?' 검색 결과 (DB 내부 직접 쿼리):
문서: **붉은사슴 뿔버섯 섭취 시, 독소로 인해 급성 장염 증상(복통, 설사, 구토, 발열 등)이 나타납니다.**
유사도 거리: 0.5172
---------------------------------------


In [ ]:
# RAG 대화 시스템이 시작되었습니다. '종료' 또는 'exit'를 입력하면 종료됩니다.

while True:
    query = input("\n문장을 입력하세요 (종료/exit 입력 시 종료): ") # 질문이 아닌 '문장'으로 변경
    if query.lower() == '종료' or query.lower() == 'exit':
        print("RAG 대화 시스템을 종료합니다.")
        break

    # 1. 문장을 임베딩 모델로 벡터화 (이전과 동일)
    query_embedding = embedding_model.encode([query]).tolist()

    # 2. 벡터 데이터베이스에서 가장 유사한 문서 검색
    # 유사도 거리(distances)를 함께 반환받도록 include=['documents', 'distances'] 추가
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=2, # 검색 결과 수 조절
        include=['documents', 'distances']
    )
    retrieved_documents = results['documents'][0]
    retrieved_distances = results['distances'][0] # 검색된 문서들의 유사도 거리

    print("\n--- 검색된 관련 문서 ---")
    if retrieved_documents:
        for i, doc in enumerate(retrieved_documents):
            # 검색된 문서와 그 유사도 거리를 함께 출력하여 확인
            print(f"문서 {i+1}: {doc} (유사도 거리: {retrieved_distances[i]:.4f})")
    else:
        print("관련 문서를 찾을 수 없습니다.")

    # --- 새로 추가된 도메인 관련성 판단 로직 ---
    is_medical_related = False
    # 유사도 임계값 설정 (이 값을 조절하여 관련성 판단 기준을 변경할 수 있습니다.)
    # all-MiniLM-L6-v2의 L2 distance 기준, 0에 가까울수록 매우 유사, 1.0~1.5 정도가 일반적인 '유사' 범위.
    # 이보다 크면 관련성이 낮다고 판단할 수 있습니다.
    relevance_threshold = 1.2

    if retrieved_documents and retrieved_distances[0] < relevance_threshold:
        # 검색된 문서가 있고, 가장 유사한 문서의 거리가 임계값보다 작으면 관련성 높음
        is_medical_related = True
    else:
        # 검색된 문서가 없거나, 가장 유사한 문서의 거리가 임계값을 넘으면 관련성 낮음
        is_medical_related = False

    # 도메인 관련성에 따라 응답 분기
    if not is_medical_related:
        final_answer = "이 문장은 저희 시스템의 의료 관련 정보와 직접적인 관련이 없어서 정확히 판단하기 어렵습니다. 의료 관련 질문이나 문장을 입력해주세요."
        print(f"\n--- 최종 답변 ---")
        print(final_answer)
        continue # LLM 호출 및 이후 로직을 건너뛰고 다음 반복으로 넘어감


    # --- 3. LLM에 전달할 프롬프트 구성 (문제가 의료 관련일 경우에만 실행) ---
    context_str = "\n".join(retrieved_documents)

    # 이 시점에서는 is_medical_related가 True이므로, 문맥이 없다는 분기는 실제로는 잘 작동하지 않을 수 있습니다.
    # 하지만 안전을 위해 남겨둡니다.
    if not context_str:
        messages = [
            {"role": "system", "content": "너는 주어진 문장이 맞는지 틀리는지 판단하고 설명하는 챗봇이야. 참고 정보가 없다면 너의 지식을 바탕으로 설명하거나 판단이 어렵다고 말해줘."},
            {"role": "user", "content": f"다음 문장이 맞는지 틀리는지 판단하고 설명해줘: {query}"}
        ]
    else:
        messages = [
            {"role": "system", "content": """너는 주어진 참고 정보를 바탕으로 사용자의 문장이 **맞는 문장인지, 틀린 문장인지 추론하고 설명하는 챗봇**이야. 특히, 어떤 물질의 섭취가 인체에 미치는 **영향이나 결과**에 초점을 맞춰 판단하고 설명해줘.
* 만약 문장이 주어진 정보와 일치하면 '맞는 문장입니다. (이유)'라고 설명해줘.
* 만약 문장이 주어진 정보와 모순되거나 틀린 내용을 포함하면 '틀린 문장입니다. (이유 및 올바른 정보)'라고 설명해줘.
* 만약 문장의 진위 여부를 주어진 정보만으로는 판단할 수 없다면 '주어진 정보로는 판단하기 어렵습니다.'라고 말해줘."""},
            {"role": "user", "content": f"참고 정보:\n{context_str}\n\n위 참고 정보를 바탕으로 다음 문장이 맞는지 틀리는지 판단하고 설명해줘: {query}"}
        ]

    prompt_for_llm = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # 4. LLM을 사용하여 답변 생성 (생성 파라미터는 동일하게 유지)
    print("--- LLM 답변 생성 중 ---")
    llm_response = generator(
        prompt_for_llm,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )

    generated_text = llm_response[0]['generated_text']

    # 답변 파싱 로직 (이전과 동일)
    final_answer = generated_text.replace(prompt_for_llm, "").strip()
    undesired_prefixes = [
        "Assistant:",
        "Assistant: 2 (Human) - You are a medical AI assistant.",
        "0 (Please respond with one of the following: 0, 1)",
        "0 (human)",
    ]
    for prefix in undesired_prefixes:
        if final_answer.startswith(prefix):
            final_answer = final_answer.replace(prefix, "", 1).strip()

    final_answer = final_answer.replace("<|eot_id|>", "").strip()
    final_answer = final_answer.replace("<s>", "").strip()
    final_answer = final_answer.replace("</s>", "").strip()

    print(f"\n--- 최종 답변 ---")
    print(final_answer)

print("\nRAG 대화 종료.")


문장을 입력하세요 (종료/exit 입력 시 종료): 붉은 사슴 뿔버섯은 장염을 일으킨다.

--- 검색된 관련 문서 ---
문서 1: 이 버섯 중독은 단순 장염 증상뿐만 아니라, 백혈구 감소증, 신부전 등 심각한 전신 증상을 초래하며 사망에 이를 수도 있습니다. (유사도 거리: 0.4607)
문서 2: 장염은 위와 소장의 염증을 의미합니다. **장염은 세균, 바이러스, 독소, 기생충 또는 특정 식품 섭취 등으로 발생할 수 있습니다.** (유사도 거리: 0.6050)
--- LLM 답변 생성 중 ---

--- 최종 답변 ---
틀린 문장입니다. (이유 및 올바른 정보) 
붉은 사슴 뿔버섯은 장염을 일으키는 식품이 아닙니다. 장염은 세균, 바이러스, 독소, 기생충 또는 특정 식품 섭취 등으로 발생할 수 있습니다.


KeyboardInterrupt: Interrupted by user